<a href="https://colab.research.google.com/github/rajarameshchindu/datascience/blob/master/Write_Better_Python_Functions_Using_Type_Dispatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2021/04/write-better-python-functions-using-type-dispatch/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29

In [1]:
# Import required libraries
import numpy as np
import torch
from PIL import Image as PILImage

# Set Torch device to GPU if CUDA supported GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to convert image to Torch tensor
def Tensor(inp):
    
    # Read image from disk using PIL Image
    img = PILImage.open(inp).convert('RGB')
    
    # Convert the image to numpy ndarray
    imgArr = np.asarray(img, np.uint8)
    
    # Rearrange the shape of the array so that it is pytorch compatible
    imgArr = imgArr.transpose(2, 0, 1)
    
    # Convert Numpy array to Torch Tensor and move it to device
    return torch.from_numpy(imgArr).to(device)

In [2]:
# Download a sample image to disk
import urllib.request 

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/200px-Python-logo-notext.svg.png"
filename = "sample.png"
urllib.request.urlretrieve(url, filename)

('sample.png', <http.client.HTTPMessage at 0x7f834fb61550>)